In [ ]:
from sf_quant.data.exposures import load_exposures
from sf_quant.data.assets import load_assets
from sf_quant.data.factors import load_factors

import datetime as dt

from factor_momentum._constants import FACTORS

import polars as pl

In [ ]:
start = dt.date(2000, 1, 1)
end = dt.date(2024, 1, 1)

In [ ]:
columns = ['date', 'barrid'] + FACTORS

daily = load_assets(start=start, end=end, columns=['date', 'barrid', 'return', 'market_cap'], in_universe=True).join(
        load_exposures(start=start, end=end, columns=columns, in_universe=True),
        on=['barrid', 'date'],
        how='inner'
    ).sort(['barrid', 'date'])

df = (daily.lazy().with_columns(
        pl.col('date').dt.truncate('1mo').alias('month')
    )
    .group_by(['month', 'barrid']).agg(
        [(np.log(1 + pl.col('return')*.01).sum())
        .alias('ret'),
        pl.col('market_cap').last()]
        +
        [pl.col(fac).mean() for fac in FACTORS]
    )
    .sort(['barrid', 'month'])
    .collect()
    )